In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms
from torchvision.transforms import functional
from PIL import ImageStat

from keras.models import load_model
import tensorflow as tf
import cv2


from PIL import Image, ImageOps
from pylab import rcParams
from typing import cast, Any, Union, Dict, List
import time
import random
import copy 
from rembg import remove
from typing import cast, Any, Union, Dict, List
 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import os
import shutil
from collections import OrderedDict

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [2]:
class FashionDataset(Dataset):
    def __init__(self, data, transform=None):        
        self.fashion = list(data.values)
        self.transform = transform
        
        label, image = [], []
        
        for i in self.fashion:
            label.append(i[0])
            image.append(i[1:])
        self.labels = np.asarray(label)
        self.images = np.asarray(image).reshape(-1, 28, 28).astype('float32')
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        label = self.labels[idx]
        image = self.images[idx]      
        
        if self.transform is not None:
            # transfrom the numpy array to PIL image before the transform function
            pil_image = Image.fromarray(np.uint8(image)) 
            image = self.transform(pil_image)
            
        return image, label

In [3]:
train_csv = pd.read_csv(r"DatasetV2\train\Hexacore-Train.csv")
test_csv = pd.read_csv(r"DatasetV2\test\Hexacore-Test.csv")

In [4]:
## Data Mnist Image folder
train_transform1 = transforms.Compose([
    transforms.RandomChoice([
        transforms.Compose([transforms.RandomVerticalFlip(),transforms.RandomHorizontalFlip()]),
        transforms.Compose([transforms.RandomHorizontalFlip(),transforms.RandomVerticalFlip()])
    ]),
    transforms.RandomAffine(degrees=30,translate=(0.1,0.1)),
    transforms.Resize((70,70)),
    transforms.ToTensor()
    ])

train_transform2 = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.Resize((70,70)),
    transforms.ToTensor()
])

train_transform3 = transforms.Compose([
    transforms.Resize((70,70)),
    transforms.ToTensor()
])

train_transform4 = transforms.Compose([
    transforms.Resize((70,70)),
    transforms.ToTensor(),
    transforms.RandomErasing(p=1, scale=(0.02, 0.2))
])

test_transform = transforms.Compose([
    transforms.Resize((70,70)),
    transforms.ToTensor()
])

In [5]:
# Import data
train_set1 = FashionDataset(train_csv,transform=train_transform1)
train_set2 = FashionDataset(train_csv,transform=train_transform2)
train_set3 = FashionDataset(train_csv,transform=train_transform3)
train_set4 = FashionDataset(train_csv,transform=train_transform4)
train_sets = torch.utils.data.ConcatDataset([train_set1,train_set2,train_set3,train_set4])

test_set = FashionDataset(test_csv,transform=test_transform)
train_set, val_set = torch.utils.data.random_split(train_sets, [int(len(train_sets) * 0.9),int(len(train_sets) * 0.1)])

train_dataloader = torch.utils.data.DataLoader(train_set,
                                               batch_size = 100,
                                               shuffle = True)
val_dataloader = torch.utils.data.DataLoader(val_set,
                                             batch_size=100)

loaders = {
    'train' : train_dataloader,
    'val' : val_dataloader
}

# Model

In [6]:
class VGG(nn.Module):
  def __init__(self, features:nn.Module, num_classes:int=1000, init_weights:bool=True):
      super(VGG, self).__init__()
      self.features = features #features = Feature extraction
      self.avgpool = nn.AdaptiveAvgPool2d((1,1))
      self.classifier = nn.Sequential(
          nn.Linear(1024, 1024),
          nn.ReLU(),
          nn.Dropout(0.5),
          nn.Linear(1024, 512),
          nn.ReLU(),
          nn.Dropout(0.3),
          nn.Linear(512, 512),
          nn.ReLU(),
          nn.Linear(512, num_classes)
          )

      if init_weights:
        self.initialize_weights()

  def forward(self, x):
    x = self.features(x) #features = Feature extraction
    x = self.avgpool(x)
    # print(x.shape)
    x = torch.flatten(x, 1)
    x = self.classifier(x)
    return x

  def initialize_weights(self):
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='leaky_relu')
        if m.bias is not None:
          nn.init.constant_(m.bias, 0)
      if isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
      if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='leaky_relu')
        if m.bias is not None:
          nn.init.constant_(m.bias, 0)

cfgs: Dict[str, List[Union[int, str]]] = {
    'A': [64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 1024, 1024, 1024, 'M'],
    'B': [64, 'M', 128, 'M', 256, 'M', 512, 'M', 1024, 'M'],
    'firstPadding':2
}

def make_layers(cfg:List[Union[int,str]], batch_norm:bool=False) -> nn.Sequential:
  layers:List[nn.Module] = []
  in_channels = 3
  in_padding = 5
  i = 0
  for v in cfg:
    if v == 'M':
      layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
    else:
      v = cast(int, v)
      in_padding = 1
      if i == 5:
        in_padding = 2
      conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=in_padding)
      if batch_norm:
        layers += [conv2d, nn.BatchNorm2d(v), nn.PReLU(num_parameters=1)]
      else:
        layers += [conv2d, nn.PReLU(num_parameters=1)] #nn.PReLU(num_parameters=1) nn.ReLU(inplace=True)
      in_channels = v
    i += 1
  return nn.Sequential(*layers)

def selfDefineVgg(arch, cfg, batch_norm,  num_classes:int, **kwargs: Any) -> VGG:
    model = VGG(make_layers(arch[cfg], batch_norm=batch_norm), num_classes, **kwargs)
    return model

In [7]:
model = selfDefineVgg(cfgs, 'A', True, 11)
model = model.to(device)

In [8]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=1)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): PReLU(num_parameters=1)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): PReLU(num_parameters=1)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): PReLU(num_parameters=1)
  